<a href="https://www.kaggle.com/code/muhammadmeftahmafazy/classification?scriptVersionId=129659743" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Mengimpor *Library*

In [ ]:
# feature extractoring and preprocessing data
import librosa
import librosa, librosa.display, os, csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_curve,roc_auc_score, auc, plot_roc_curve
from sklearn import metrics
from sklearn.svm import SVC

#Pytorch
import torch

from imblearn.pipeline import Pipeline

import joblib

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#Other
from IPython.display import Audio
from tqdm import tqdm

# Inisial Dataset

In [ ]:
#Panggil Dataset
data = pd.read_csv("/kaggle/input/d/muhammadmeftahmafazy/covid-19-cough/mfcc20+all.csv")
data

In [ ]:
#Menghapus kolom nama file
data_new = data.drop('filename', axis=1)
data_new

In [ ]:
#Mengubah label menjadi numerik
data_new['label'].replace(['healthy', 'COVID-19'], [0,1], inplace=True)

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel. 
# array ke-16 untuk dataset 10 mfcc
# array ke-26 untuk dataset 20 mfcc
# array ke-36 untuk dataset 30 mfcc
# X = data_new[data_new.columns[:26]]

# array ke-24 untuk dataset 10 mfcc
# array ke-34 untuk dataset 20 mfcc
# array ke-44 untuk dataset 30 mfcc
X = data_new[data_new.columns[:34]]


# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y = data_new['label']

# Klasifikasi menggunakan *SVM* dengan Stratified K-Fold

In [ ]:
# standarisasi nilai-nilai dari dataset
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# Inisialisasi objek stratified k-fold dengan inisial skf
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
# # Loop untuk setiap fold pada data
# for train_index, test_index in skf.split(X, y):
#     # Memisahkan data untuk training dan testing
#     X_train, X_test = X.values[train_index], X.values[test_index]
#     y_train, y_test = y.values[train_index], y.values[test_index]

# Mencari Parameter Terbaik menggunakan algoritma GridSearchCV

In [ ]:
# # Mencari parameter terbaik menggunakan Grid Search
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# param_grid = {'C': [0.01, 0.1, 1, 10, 100], 
#               'gamma': [0.01, 0.1, 1, 10, 100], 
#               'kernel': ['rbf'],
#              }

# clf = GridSearchCV(SVC(), param_grid, cv=skf, scoring='roc_auc')
# clf.fit(X_train_scaled, y_train)

# # Menampilkan parameter terbaik dan AUC pada data testing
# y_pred = clf.decision_function(X_test_scaled)
# auc = roc_auc_score(y_test, y_pred)

# print("Best parameters: ", clf.best_params_)
# print("AUC on test set: ", auc)

In [ ]:
# # Inisialisasi objek SVM
# clf = SVC(kernel='rbf')
# clf = SVC (kernel='rbf', C=1, gamma='scale', probability=True)
clf = SVC(kernel='rbf', C=1, gamma=0.1, probability=True)   
    
# Loop untuk setiap fold pada data
for train_index, test_index in skf.split(X_scaled, y):
        # Memisahkan data untuk training dan testing
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Melakukan training pada model SVM
        clf.fit(X_train, y_train)

        # Melakukan prediksi pada data testing
        y_pred = clf.predict(X_test)

        # Hitung Probabilitas
        y_prob = clf.decision_function(X_test)

In [ ]:
# # Mengetahui jumlah data training dan data testing
print(len(X_train))
print(len(X_test))
print("training",pd.value_counts(y_train))
print("testing",pd.value_counts(y_test))

In [ ]:
# Menampilkan Akurasi data training dan data testing
print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

# Confusion Matrix

In [ ]:
    # Confusion Matrix
    results = []
    
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
    print('Confusion matrix\n\n',  confusion_matrix)

    print('\nTrue Positives(TP) = ',  confusion_matrix[1,1])

    print('\nTrue Negatives(TN) = ',  confusion_matrix[0,0])

    print('\nFalse Positives(FP) = ',  confusion_matrix[0,1])

    print('\nFalse Negatives(FN) = ',  confusion_matrix[1,0])

    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

    cm_display.plot()
    plt.show()
     
    Accuracy = metrics.accuracy_score(y_test, y_pred)
    Precision = metrics.precision_score(y_test, y_pred)
    Sensitivity_recall = metrics.recall_score(y_test, y_pred)
    Specificity = metrics.recall_score(y_test, y_pred, pos_label=0)

#     print(classification_report(y_test, y_pred))
    plot_roc_curve(clf, X_test, y_test)
    plt.show()
    
    # AUC (y_prob[:,1] khusus untuk random forest dan knn)
    auc = roc_auc_score(y_test, y_prob)
    
     # Simpan hasil ke dalam list
    results.append({'Accuracy': Accuracy,"Precision":Precision,"Sensitivity_recall":Sensitivity_recall,"Specificity":Specificity, 'AUC':auc})
    
    # Buat dataframe dari hasil
    df = pd.DataFrame(results)

    # Print dataframe
    print(df)

In [ ]:
    # Confusion Matrix
    print(classification_report(y_test, y_pred))